In [1]:
import time
import numpy as np
import pandas as pd
import os
import sys
import sklearn
import datetime
import random
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization, GlobalAveragePooling2D  
from tensorflow.keras.backend import batch_normalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from packaging import version
%matplotlib inline

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
#identify GPU
device_name = tf.test.gpu_device_name()
if not tf.test.is_gpu_available():
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Found GPU at: /device:GPU:0


In [3]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.1.0


In [4]:
#train data
X_train = np.load(os.path.join("Data_new3", "X_train.npy"))
y_train = np.load(os.path.join("Data_new3", "train_labels_multi.npy"))
y_train_bi = np.load(os.path.join("Data_new3", "y_train.npy"))

#test data
X_test = np.load(os.path.join("Data_new3", "X_test.npy"))
y_test = np.load(os.path.join("Data_new3", "y_test_labels_multi.npy"))
y_test_bi = np.load(os.path.join("Data_new3", "y_test.npy"))

#validation data
X_val = np.load(os.path.join("Data_new3", "X_val.npy"))
y_val = np.load(os.path.join("Data_new3", "y_val_labels_multi.npy"))
y_val_bi = np.load(os.path.join("Data_new3", "y_val.npy"))

In [5]:
#train data
print("X_train data:", X_train.shape)
print("y_train data:", y_train.shape)

X_train data: (2913, 512, 512, 1)
y_train data: (2913,)


In [6]:
print(y_train.shape)

(2913,)


In [7]:
#validation data
print("X_validation data:", X_val.shape)
print("y_validation data:", y_val.shape)

X_validation data: (327, 512, 512, 1)
y_validation data: (327,)


In [8]:
#test data
print("X_test data:", X_test.shape)
print("y_test data:", y_test.shape)

X_test data: (328, 512, 512, 1)
y_test data: (328,)


In [9]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

#binary
y_train_bi = to_categorical(y_train_bi)
y_val_bi = to_categorical(y_val_bi)
y_test_bi = to_categorical(y_test_bi)

In [10]:
# scale pixels
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

In [11]:
y_val.shape

(327, 4)

In [12]:
classes = 4
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=X_train.shape[1:]))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(classes, activation='softmax'))
    # compile model
    opt = Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
model = define_model()

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 512, 512, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 256, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 2097152)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               268435584 
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 268,445,668
Trainable params: 268,445,668
Non-trainable params: 0
__________________________________________

In [16]:
# fit model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

Train on 2913 samples, validate on 327 samples
Epoch 1/30
  32/2913 [..............................] - ETA: 31:11

ResourceExhaustedError:  OOM when allocating tensor with shape[2097152,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MatMul_3 (defined at <ipython-input-15-e4b039c1f9a0>:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_894]

Function call stack:
distributed_function


In [ ]:
# fit model
history = model.fit(X_val, y_val, epochs=1, batch_size=32)

In [ ]:
_, acc = model.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))